# Booking.com TrustPilot Project
This project is based on the tutorial provided by the ScraperAPI in this [link](https://www.scraperapi.com/blog/scraping-trustpilot-reviews/).


In [2]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import csv

In [ ]:
company = "booking.com"
base_url = f"https://www.trustpilot.com/review/www.booking.com"

headers = {
	'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
	'accept-language': 'en-US,en;q=0.9'
}
payload = {
	'api_key': "YOUR API KEY",
	'url': base_url,
	'render': 'true',
	'keep_headers': 'true',
}

try:
	response = requests.get('https://api.scraperapi.com', params=payload, headers=headers)
	soup = BeautifulSoup(response.content, 'html.parser')
	pages_to_scrape = 10
	
	with open('trustpilot_reviews.csv', 'w', newline='', encoding='utf-8') as csvfile:
		csv_writer = csv.writer(csvfile)
		csv_writer.writerow(['Reviewer', 'Rating', 'Review', 'Date'])
		
		for page in range(1, pages_to_scrape):
			payload['url'] = f"{base_url}?page={page}"
			page_response = requests.get('https://api.scraperapi.com', params=payload, headers=headers)
			page_soup = BeautifulSoup(page_response.content, 'html.parser')
			reviews = page_soup.find_all('div', {"class": "styles_reviewCardInner__EwDq2"})
			
			for review in reviews:
				reviewer = review.find("span", attrs={"class": "typography_heading-xxs__QKBS8"}).text
				rating = review.find("div", attrs={"class": "styles_reviewHeader__iU9Px"})["data-service-review-rating"]
				content_element = review.find("p", attrs={"class": "typography_body-l__KUYFJ"})
				content = content_element.text if content_element else 'None'
				date = review.find("p", attrs={"class":"typography_body-m__xgxZ_ typography_appearance-default__AAY17"}).text
				csv_writer.writerow([reviewer, rating, content, date])
	print("Data Extraction Successful!")
except Exception as e:
	print("An error occurred:", e)